# Day 14: Docking Data

As your ferry approaches the sea port, the captain asks for your help again. The computer system that runs this port isn't compatible with the docking program on the ferry, so the docking parameters aren't being correctly initialized in the docking program's memory.

After a brief inspection, you discover that the sea port's computer system uses a strange bitmask system in its initialization program. Although you don't have the correct decoder chip handy, you can emulate it in software!

The initialization program (your puzzle input) can either update the bitmask or write a value to memory. Values and memory addresses are both 36-bit unsigned integers. For example, ignoring bitmasks for a moment, a line like mem[8] = 11 would write the value 11 to memory address 8.

The bitmask is always given as a string of 36 bits, written with the most significant bit (representing 2^35) on the left and the least significant bit (2^0, that is, the 1s bit) on the right. The current bitmask is applied to values immediately before they are written to memory: a 0 or 1 overwrites the corresponding bit in the value, while an X leaves the bit in the value unchanged.

For example, consider the following program:

```text
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
```

This program starts by specifying a bitmask (mask = ....). The mask it specifies will overwrite two bits in every written value: the 2s bit is overwritten with 0, and the 64s bit is overwritten with 1.

The program then attempts to write the value 11 to memory address 8. By expanding everything out to individual bits, the mask is applied as follows:

```tezt
value:  000000000000000000000000000000001011  (decimal 11)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001001001  (decimal 73)
```

So, because of the mask, the value 73 is written to memory address 8 instead. Then, the program tries to write 101 to address 7:

```text
value:  000000000000000000000000000001100101  (decimal 101)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001100101  (decimal 101)
```

This time, the mask has no effect, as the bits it overwrote were already the values the mask tried to set. Finally, the program tries to write 0 to address 8:

```text
value:  000000000000000000000000000000000000  (decimal 0)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001000000  (decimal 64)
```

64 is written to address 8 instead, overwriting the value that was there previously.

To initialize your ferry's docking program, you need the sum of all values left in memory after the initialization program completes. (The entire 36-bit address space begins initialized to the value 0 at every address.) In the above example, only two values in memory are not zero - 101 (at address 7) and 64 (at address 8) - producing a sum of 165.

Execute the initialization program. What is the sum of all values left in memory after it completes?

In [1]:
# Python imports
from collections import defaultdict
from itertools import combinations_with_replacement
from pathlib import Path
from typing import List, Tuple, Union

import numpy as np

As usual, we write an input parser. Here the input is a program that has two kinds of statements: a *mask* and an *(address, value)* tuple, so we treat these separately, but load them into a single list of program statements.

The *mask* is defined straightforwardly as a string, so we use this.

The memory *address* is presented as an integer, but the *value* is an integer, which we need to convert to binary in order to mask correctly (Python has a `bin()` function to help here), and we left-pad the result to a 36-bit binary value, as a string.

In [2]:
def load_program(fpath: str) -> List[Union[str, Tuple[int, str]]]:
    """Returns program loaded from passed file
    
    :param fpath:  path to input program
    """
    program = []  # Holds the parsed program
    
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            if line.startswith("mask"):
                program.append(line.strip().split()[-1])
            elif line.startswith("mem"):
                line = line.split()
                mem, val = line[0][:-1], line[-1]
                memloc = int(mem.split("[")[-1])
                val = bin(int(val))[2:]
                program.append((memloc, f"{int(val):036}"))
            
    return program

In [3]:
program = load_program("day14_test.txt")
program

['XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X',
 (8, '000000000000000000000000000000001011'),
 (7, '000000000000000000000000000001100101'),
 (8, '000000000000000000000000000000000000')]

Now, when running the program with `run_program()`, we keep a *memory* which is a dictionary keyed by memory address. This holds `int` values at each specified address.

For each line in the program we either load in a mask as a list of `(index, value)` tuples, so that we can apply the mask to the binary representation in other program statements, by changing the bit at `index` to `value`.

For statements that update a memory location, we take the binary representation, convert it to a `list` and update the bit at `index` to `value` according to the current mask. We then convert the binary value to an int (`int(value, 2)` does this - the `2` indicates the base being used), and update the corresponding memory location.

Finally, we return the sum of values in all memory addresses.

In [4]:
def run_program(program: List[Union[str, Tuple[int, str]]]) -> int:
    """Returns sum of values in memory after running passed program
    
    :param program:  input mask and (address, value) tuples
    """
    memory = defaultdict(int)

    for line in program:
        if isinstance(line, str):  # The program statement is a new mask
            maskvals = []
            for idx, val in enumerate(line):
                if val != "X":
                    maskvals.append((idx, val))
        elif isinstance(line, tuple):  # The program statement is an address and value
            loc, val = line[0], list(line[1])
            for idx, mval in maskvals:
                val[idx] = mval
            maskedval = int("".join(val), 2)
            memory[loc] = maskedval
    
    return sum(memory.values())

In [5]:
program = load_program("day14_test.txt")
run_program(program)

165

In [6]:
program = load_program("day14_data.txt")
run_program(program)

14925946402938

Part Two

For some reason, the sea port's computer system still can't communicate with your ferry's docking program. It must be using version 2 of the decoder chip!

A version 2 decoder chip doesn't modify the values being written at all. Instead, it acts as a memory address decoder. Immediately before a value is written to memory, each bit in the bitmask modifies the corresponding bit of the destination memory address in the following way:

    If the bitmask bit is 0, the corresponding memory address bit is unchanged.
    If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
    If the bitmask bit is X, the corresponding memory address bit is floating.

A floating bit is not connected to anything and instead fluctuates unpredictably. In practice, this means the floating bits will take on all possible values, potentially causing many memory addresses to be written all at once!

For example, consider the following program:

```text
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
```

When this program goes to write to memory address 42, it first applies the bitmask:

```text
address: 000000000000000000000000000000101010  (decimal 42)
mask:    000000000000000000000000000000X1001X
result:  000000000000000000000000000000X1101X
```

After applying the mask, four bits are overwritten, three of which are different, and two of which are floating. Floating bits take on every possible combination of values; with two floating bits, four actual memory addresses are written:

```text
000000000000000000000000000000011010  (decimal 26)
000000000000000000000000000000011011  (decimal 27)
000000000000000000000000000000111010  (decimal 58)
000000000000000000000000000000111011  (decimal 59)
```

Next, the program is about to write to memory address 26 with a different bitmask:

```text
address: 000000000000000000000000000000011010  (decimal 26)
mask:    00000000000000000000000000000000X0XX
result:  00000000000000000000000000000001X0XX
```

This results in an address with three floating bits, causing writes to eight memory addresses:

```text
000000000000000000000000000000010000  (decimal 16)
000000000000000000000000000000010001  (decimal 17)
000000000000000000000000000000010010  (decimal 18)
000000000000000000000000000000010011  (decimal 19)
000000000000000000000000000000011000  (decimal 24)
000000000000000000000000000000011001  (decimal 25)
000000000000000000000000000000011010  (decimal 26)
000000000000000000000000000000011011  (decimal 27)
```

The entire 36-bit address space still begins initialized to the value 0 at every address, and you still need the sum of all values left in memory at the end of the program. In this example, the sum is 208.

Execute the initialization program using an emulator for a version 2 decoder chip. What is the sum of all values left in memory after it completes?

To accommodate the changes for the version 2 decoder chip, we read in the mask as before, but now take the (address, value) tuples as an int and a string of the decimal number, not the binary number.

In [7]:
def load_program_v2(fpath: str) -> List[Union[str, Tuple[int, str]]]:
    """Return a v2 program from the passed file
    
    :param fpath:  path to input program file
    """
    program = []
    
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            if line.startswith("mask"):
                program.append(line.strip().split()[-1])
            elif line.startswith("mem"):
                line = line.split()
                mem, val = line[0][:-1], line[-1]
                memloc = int(mem.split("[")[-1])
                program.append((memloc, val))
            
    return program

In [8]:
load_program_v2("day14_test_v2.txt")

['000000000000000000000000000000X1001X',
 (42, '100'),
 '00000000000000000000000000000000X0XX',
 (26, '1')]

In [9]:
def run_v2(program: List[Union[str, Tuple[int, str]]]) -> int:
    """Return sum of values in memory after running the passed program
    
    :param program:  a v2 program
    """
    memory = defaultdict(int)

    for line in program:
        if isinstance(line, str):
            maskvals = []
            for idx, val in enumerate(line):
                if val != "0":
                    maskvals.append((idx, val))
        elif isinstance(line, tuple):
            newlocs = []
            
            loc, val = line[0], int(line[1])
            locbin = list(f"{int(bin(int(loc))[2:]):036}")
        
            # change 1s
            for idx, mval in maskvals:
                if mval == "1":
                    locbin[idx] = mval

            # change Xs
            xidx = [idx for (idx, val) in maskvals if val == "X"]
            for binmask in [list(f"{int(bin(_)[2:]):0{len(xidx)}}") for _ in range(2**len(xidx))]:
                newmask = locbin.copy()
                for idx, bval in zip(xidx, binmask):
                    newmask[idx] = bval
                newaddr = int("".join(newmask), 2)
                memory[newaddr] = val
                
    return sum(memory.values())

In [10]:
program = load_program_v2("day14_test_v2.txt")
run_v2(program)

208

In [11]:
program = load_program_v2("day14_data.txt")
run_v2(program)

3706820676200